In [4]:
import pyspark
import requests
import json 

In [7]:
def get_api_endpoint(path:str,filter:str):

    response = requests.get(f"{path}?{filter}")

    if response.status_code == 200:
        print(response.text)
        return json.loads(response.text)
    else:
        return response.status_code


In [8]:
path = "https://dataestur.azure-api.net/API-SEGITTUR-v1/AENA_DESTINOS_DL"
filter = "desde%20%28a%C3%B1o%29=2004&Aeropuerto%20AENA=JT%20Barcelona-El%20Prat"
data = get_api_endpoint(path,filter)

PK   0�ZAMb�   �      docProps/app.xmlM�=1D��q��A�Bb@�R��{/�dC�B~�9��noF�
�M��<,���EG��8y��.b膈��x`�/�ֻ�/��W2$A0����n�5y���SeQ���Ί�=/�E�~�\?�oB������x(	��M}PK   0�Z�\�#  �'     xl/theme/theme1.xml�Z[s�8~��xg�m�6���siv۴���N�X�lyd���G6˖
;:�X�v|��ퟌ��t4m���x8��ҋp�Q���l��A	��i�d��ڮ����SO��}��h��e�WA&�����l����>e��:2�n0X �o��NZ��T���jg?Vk���H���}��I���2
��O�î�ƫ�t�i&'��k��hBF��z��@�  Xpv����^)�u����A\�X�9�����i��4Fr�� 7��LP|�A����\���)�P�ȁ�G�!�ܯ����ɤ3z�}:�k�i�����ϓ�s�䟧��MB�p�,	��#[a�'n;r:gB|�����%2���
�9~MH�+�����@�/$�J��i�#�t&��3�F�u�h�<z���5�F��U�ܣ�X��4�5,��x���<͔A���N<gV��]��䞌r#���X}�OGn#ש��"הF$E��-��8�I
�g�F!�i���$����+B>b6r���q��`Z��xNҴ�Y�5�>`��͑u�֑�^7B>b΋���8J���qX��^�I���f��~��3l,��G�J�&�?�24��Y	��Vj���4>�2
�>�zx
7�ƼP��{���7«���9.}ϥ��=�ҷ7#}g�Ӌ[�Fn[���1��4.(cWr���T��)�9����h>�����$���Y-#�K��A$�����*�	�d[%	�T�e7��Bn�S�J��对(�<[�鯡t>,��<_��3C�rK궔��&8J���pN�;g<��w�5��]v�#�0S�C�B�m���

JSONDecodeError: Expecting value: line 1 column 1 (char 0)